In [1]:
import json, os, pandas as pd, pymongo, spacy, string

/home/maaz-lfd/anaconda3/envs/py38/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(
/home/maaz-lfd/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-06 21:51:09.952006: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 21:51:20.886816: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dl

In [3]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")

db = mongo_client["cord_literature"]
col_ct = db['clinical_trials']
col_processed_ct = db['processed_clinical_trials']


In [2]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")

db = mongo_client["cord_literature"]
col_ct = db['round_5_clinical_trials']
col_processed_ct = db['round_5_processed_clinical_trials']


In [3]:
col_ct

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'cord_literature'), 'round_5_clinical_trials')

In [5]:
dataset_path = '/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5'
clinical_path_pmc = os.path.join(dataset_path,'pmc')

In [6]:
CT_dt = pd.read_csv(os.path.join(dataset_path,'round_5_CT_metdata.csv'))

In [7]:
len(CT_dt)

9119

In [8]:
CT_dt['abstract'] = CT_dt['abstract'].fillna('NA')

In [9]:
# populating paper ID, PUBLISH TIME, JOURNAL, AUTHORS, TITLE, BODY in clinical_trials collection
for index, rows in CT_dt.iterrows():
    pmc_data = json.load(open(os.path.join(clinical_path_pmc,rows['pmcid']+'.xml.json')))
    if rows['abstract'] != 'NA':
        mongo_dict = {'_id':rows['pmcid'], 'publish_time':rows['publish_time'], 'journal':rows['journal'], 'authors':rows['authors'], 'title':rows['title'], 'abstract':rows['abstract']}
    else:
        print(rows['pmcid'])
        mongo_dict = {'_id':rows['pmcid'], 'publish_time':rows['publish_time'], 'journal':rows['journal'], 'authors':rows['authors'], 'title':rows['title'],}
    mongo_dict.update({i['section']: i['text'] for i in pmc_data['body_text']})
    col_ct.insert_one(mongo_dict)

PMC3219342
PMC3219404
PMC3291847
PMC3567830
PMC3772528
PMC4028196
PMC4056101
PMC4613429
PMC4829481
PMC5253181
PMC5581326
PMC5718958
PMC5853239
PMC6160391
PMC6289537
PMC6378949
PMC6770994
PMC7026208
PMC7079835
PMC7079918
PMC7080677
PMC7091246
PMC7091610
PMC7092369
PMC7094241
PMC7094608
PMC7094909
PMC7095014
PMC7095093
PMC7095109
PMC7095199
PMC7095202
PMC7095221
PMC7095223
PMC7095248
PMC7095358
PMC7095429
PMC7095915
PMC7097414
PMC7097419
PMC7102200
PMC7102668
PMC7102817
PMC7103464
PMC7104931
PMC7106314
PMC7107922
PMC7108197
PMC7110210
PMC7114683
PMC7115724
PMC7118647
PMC7119064
PMC7119315
PMC7120466
PMC7123340
PMC7123786
PMC7124193
PMC7124212
PMC7125967
PMC7127031
PMC7127542
PMC7127870
PMC7128141
PMC7128167
PMC7128223
PMC7128303
PMC7128330
PMC7128362
PMC7128500
PMC7128690
PMC7128945
PMC7128997
PMC7129102
PMC7129215
PMC7129296
PMC7129306
PMC7129368
PMC7129943
PMC7129953
PMC7130014
PMC7130432
PMC7131708
PMC7133623
PMC7133714
PMC7133842
PMC7134186
PMC7134409
PMC7134438
PMC7134497
PMC7134619

In [10]:
nlp = spacy.load("en_core_web_sm")

In [11]:
def processed_text(text):
    tok_text = [tok.lemma_.lower().strip() for tok in nlp(str(text)) if not tok.like_num and not tok.is_punct]
    tok_text = [tok for tok in tok_text if (tok not in nlp.Defaults.stop_words and tok not in string.punctuation and type(tok) != int and type(tok) != float)]
    return ' '.join(tok_text)

In [12]:
# populating processed text TITLE, BODY in processed_clinical_trials collection
populated_data = [id['_id'] for id in col_processed_ct.find({},{'_id':1})]

for index, rows in CT_dt.iterrows():
    if rows['pmcid'] not in populated_data:
        pmc_data = json.load(open(os.path.join(clinical_path_pmc,rows['pmcid']+'.xml.json')))
        prc_title = processed_text(rows['title']) 
        prc_body = {i['section']: processed_text(i['text']) for i in pmc_data['body_text']}

        # print(rows['title'])
        # print(prc_title)
        # print('--------------------------------------------------------------')
        # print(rows['abstract'])
        # print('--------------------------------------------------------------')
        # print(prc_abstract)
        # print('=================================================================')

        if rows['abstract'] != 'NA':
            prc_abstract = processed_text(rows['abstract'])
            mongo_dict = {'_id':rows['pmcid'], 'title':prc_title, 'abstract':prc_abstract}
        else:
            print(rows['pmcid'])
            mongo_dict = {'_id':rows['pmcid'], 'title':prc_title}
        
        mongo_dict.update(prc_body)
        mongo_dict.update({'all_text':' '.join(mongo_dict[key] for key in mongo_dict if key != '_id') })
        col_processed_ct.insert_one(mongo_dict)

PMC3219342
PMC3219404
PMC3291847
PMC3567830
PMC3772528
PMC4028196
PMC4056101
PMC4613429
PMC4829481
PMC5253181
PMC5581326
PMC5718958
PMC5853239
PMC6160391
PMC6289537
PMC6378949
PMC6770994
PMC7026208
PMC7079835
PMC7079918
PMC7080677
PMC7091246
PMC7091610
PMC7092369
PMC7094241
PMC7094608
PMC7094909
PMC7095014
PMC7095093
PMC7095109
PMC7095199
PMC7095202
PMC7095221
PMC7095223
PMC7095248
PMC7095358
PMC7095429
PMC7095915
PMC7097414
PMC7097419
PMC7102200
PMC7102668
PMC7102817
PMC7103464
PMC7104931
PMC7106314
PMC7107922
PMC7108197
PMC7110210
PMC7114683
PMC7115724
PMC7118647
PMC7119064
PMC7119315
PMC7120466
PMC7123340
PMC7123786
PMC7124193
PMC7124212
PMC7125967
PMC7127031
PMC7127542
PMC7127870
PMC7128141
PMC7128167
PMC7128223
PMC7128303
PMC7128330
PMC7128362
PMC7128500
PMC7128690
PMC7128945
PMC7128997
PMC7129102
PMC7129215
PMC7129296
PMC7129306
PMC7129368
PMC7129943
PMC7129953
PMC7130014
PMC7130432
PMC7131708
PMC7133623
PMC7133714
PMC7133842
PMC7134186
PMC7134409
PMC7134438
PMC7134497
PMC7134619

In [7]:
a = json.load(open(os.path.join(clinical_path_pmc,'PMC137267.xml.json')))

In [9]:
a.keys()

dict_keys(['paper_id', 'metadata', 'body_text', 'ref_entries', 'back_matter', 'bib_entries'])

In [18]:
for i in a['body_text']:
    print(i['text'])

Anaemia is a common condition in critically ill patients, and RBC transfusions are often used in the treatment and management of this patient population. In fact, one study [1] reported that 25% of all critically ill patients received RBC transfusions. Many laboratory studies [2,3,4,5,6,7,8] have examined the physiological responses (ie compensatory mechanisms) of the body to anaemia, which include the following [9]: increased cardiac output, decreased blood viscosity, capillary changes, increased oxygen extraction, and other tissue adaptations to meet oxygen requirements. Although critically ill patients are affected by a number of factors that predispose them to the adverse consequences of anaemia, persistence of this condition is of particular concern because it may cause the compensatory mechanisms in these patients to become impaired, risking oxygen deprivation in vital organs [9]. However, critically ill patients may also be at increased risk from the adverse effects of RBC trans